In [31]:
from Utils.MidiParseHandler import *
from pathlib import Path
import glob
import os
import music21 as m21

In [2]:
midi_files = glob.glob(os.path.join('Dataset-RAW', '*.mid'))

In [3]:
import concurrent.futures
import multiprocessing
import Utils  # Import the module where multi_readed is defined
import os
import shutil

# Assuming you have a list of MIDI file paths
progress_file = 'progress.txt'
output_folder = 'processed_midi_files'

num_processes = multiprocessing.cpu_count()

# Convert the data using multiprocessing with timeout
converted_data, failed_files = multiprocess_data_conversion(midi_files, num_processes)

# Save or use the converted data
# For example, save to a file or further process the converted data
# Here we just print the number of successfully converted MIDI objects
print(f"Successfully converted {len(converted_data)} MIDI files.")
print(f"Failed to convert {len(failed_files)} MIDI files.")
print(f"Failed files: {failed_files}")

all_midis = converted_data

Processed 1/152 files
Processed 2/152 files
Processed 3/152 files
Processed 4/152 files
Processed 5/152 files
Processed 6/152 files
Processed 7/152 files
Processed 8/152 files
Processed 9/152 files
Processed 10/152 files
Processed 11/152 files
Processed 12/152 files
Processed 13/152 files
Processed 14/152 files
Processed 15/152 files
Processed 16/152 files
Processed 17/152 files
Processed 18/152 files
Processed 19/152 files
Processed 20/152 files
Processed 21/152 files
Processed 22/152 files
Processed 23/152 files
Processed 24/152 files
Processed 25/152 files
Processed 26/152 files
Processed 27/152 files
Processed 28/152 files
Processed 29/152 files
Processed 30/152 files
Processed 31/152 files
Processed 32/152 files
Processed 33/152 files
Processed 34/152 files
Processed 35/152 files
Processed 36/152 files
Processed 37/152 files
Processed 38/152 files
Processed 39/152 files
Processed 40/152 files
Processed 41/152 files
Processed 42/152 files
Processed 43/152 files
Processed 44/152 fil

In [43]:
part = all_midis[0][0]

In [44]:
part.show('text')

{0.0} <music21.stream.Measure 1 offset=0.0>
    {0.0} <music21.instrument.Piano 'Piano'>
    {0.0} <music21.clef.TrebleClef>
    {0.0} <music21.tempo.MetronomeMark presto Quarter=184.0>
    {0.0} <music21.key.Key of B- major>
    {0.0} <music21.meter.TimeSignature 3/4>
    {0.0} <music21.note.Note B->
    {1.0} <music21.note.Note A>
    {1.5} <music21.note.Note B->
    {2.0} <music21.note.Note G>
{3.0} <music21.stream.Measure 2 offset=3.0>
    {0.0} <music21.note.Note B->
    {1.0} <music21.note.Note A>
    {1.5} <music21.note.Note B->
    {2.0} <music21.note.Note G>
{6.0} <music21.stream.Measure 3 offset=6.0>
    {0.0} <music21.note.Note B->
    {1.0} <music21.note.Note A>
    {1.5} <music21.note.Note B->
    {2.0} <music21.note.Note G>
{9.0} <music21.stream.Measure 4 offset=9.0>
    {0.0} <music21.note.Note B->
    {1.0} <music21.note.Note A>
    {1.5} <music21.note.Note B->
    {2.0} <music21.note.Note G>
{12.0} <music21.stream.Measure 5 offset=12.0>
    {0.0} <music21.note.Note B->

In [38]:
def list_instruments(midi):
    partStream = midi.parts.stream()
    print("List of instruments found on MIDI file:")
    for p in partStream:
        aux = p
        print (p.partName)

list_instruments(all_midis[8])

AttributeError: 'list' object has no attribute 'parts'

In [19]:
def list_songs(midi):
    parsed = []
    for score in midi:
        partStream = score.parts.stream()
        a = 0
        song = []
        for p in partStream:
            if(p.partName == "Piano"):
                song.append(p)
        parsed.append(song)
    return parsed

conv = list_songs(all_midis)

In [25]:
filtered_lists = [sublist for sublist in conv if ((len(sublist) < 3) and len(sublist) >1)]

In [27]:
len(filtered_lists)

89

In [32]:
#Helping function        
def extract_notes(file):
    notes = []
    pick = None
    for j in file:
        songs = m21.instrument.partitionByInstrument(j)
        #print(len(songs.parts))
        for part in songs.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, m21.note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, m21.chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= extract_notes(all_midis)

In [33]:
all_midis = filtered_lists 

In [34]:
from collections import Counter

count_num = Counter(Corpus)
print("Total unique notes in the Corpus:", len(count_num))

Total unique notes in the Corpus: 759


In [35]:
#Getting a list of rare chords
rare_note = []
for index, (key, value) in enumerate(count_num.items()):
    if value < 100:
        m =  key
        rare_note.append(m)
        
print("Total number of notes that occur less than 100 times:", len(rare_note))

Total number of notes that occur less than 100 times: 529


In [36]:
# Storing all the unique characters present in my corpus to bult a mapping dic. 
symb = sorted(list(set(Corpus)))

L_corpus = len(Corpus) #length of corpus
L_symb = len(symb) #length of total unique characters

#Building dictionary to access the vocabulary from indices and vice versa
mapping = dict((c, i) for i, c in enumerate(symb))
reverse_mapping = dict((i, c) for i, c in enumerate(symb))

print("Total number of characters:", L_corpus)
print("Number of unique characters:", L_symb)

Total number of characters: 417132
Number of unique characters: 759
